In [5]:
#coding=utf8
import json
import os
import numpy as np
from math import *
import time
import pandas as pd
import datetime

agis_mat = {}
#matrix_AGIS.json
for entry in json.load(open('matrix_AGIS.json', 'r')):
    agis_mat[(entry['src'], entry['dst'])] = -1
    if 'closeness' in entry:
        agis_mat[(entry['src'], entry['dst'])] = entry['closeness']

site_mapping = {}
with open('AGIS_mapping.txt', 'r') as f:
    for line in f:
        key, value = line.rstrip('\n').split()
        site_mapping[key] = value



l = os.listdir('thrdata/')
l.sort()
# thr per link
throughputs = {}
srcsites = set()
dstsites = set()
#print l[:10]
#exit(0)
print "Importing throughputs..."
s = time.time()
for filename in l:
    #print filename
    if not filename.endswith('.json'):
        continue
    throughputs[filename] = {}
    j = json.loads(open('thrdata/'+filename).read())
    transfers = j['transfers']['rows']
    sidx = j['transfers']['key']['src_site']
    didx = j['transfers']['key']['dst_site']
    bidx = j['transfers']['key']['bytes_xs']
    for t in transfers:
        # read transfers
        if t[sidx] in ['LIP-LISBON', 'UTD-HEP-SE', 'NDGF-T1-RUCIOTEST']\
                or t[didx] in ['LIP-LISBON', 'UTD-HEP-SE', 'NDGF-T1-RUCIOTEST']:
            continue
        if 'RUCIOTEST' in t[sidx]:
            #print t[sidx]
            continue
        if 'RUCIOTEST' in t[didx]:
            #print t[didx]
            continue
        try:
            src = site_mapping[t[sidx]]
        except KeyError:
            #print t[0],'not in AGIS map'
            src = t[sidx]
        try:
            dst = site_mapping[t[didx]]
        except KeyError:
            #print t[1],'not in AGIS map'
            dst = t[didx]
        # throughput in MBps
        thr = float(t[bidx])/1000/1000/3600
        srcsites.add(src)
        dstsites.add(dst)
        throughputs[filename]['%s:%s' % (src,dst)] = thr

print 'Loaded %d files in %.02f min' % (len(l),(time.time() - s)/60.)

# dis uil bi æ big mætrix
srcsites = list(srcsites)
dstsites = list(dstsites)
srcsites.sort()
dstsites.sort()
allthroughputs = {}
print "Creating big matrix"
s = time.time()
for filename in l:
    if not filename.endswith('.json'):
        continue
    y,m,d,h = filename[:-5].split('-')
    dt = datetime.datetime(int(y), int(m), int(d), int(h))
    #print dt
    keydate = time.mktime(dt.timetuple())
    for src in srcsites:
        for dst in dstsites:
            if '%s:%s' % (src, dst) not in allthroughputs:
                allthroughputs['%s:%s' % (src, dst)] = {}
            else:
                try:
                    allthroughputs['%s:%s' % (src, dst)][dt] = throughputs[filename]['%s:%s' % (src, dst)]
                except KeyError:
                    allthroughputs['%s:%s' % (src, dst)][dt] = 0.
print 'Done in %.02f min' % ((time.time() - s)/60.)
print 'creating DataFrame and writing to disk...'
s = time.time()
df = pd.DataFrame(allthroughputs)
df.to_json('allthroughputs.json')
#df.describe()
print 'Done in %.02f min' % ((time.time() - s)/60.)

Importing throughputs...
Loaded 2209 files in 0.43 min
Creating big matrix
Done in 1.61 min
creating DataFrame and writing to disk...
Done in 2.67 min
